In [1]:
import requests
import pandas as pd
import numpy as np

# 1 - Introduction

# 2 - Load dataset

## 2.1 - Create dataframes from JSON

The raw program data is available for download from the NY Philharmonic's official GitHub repository. The data is stored in `.json` format with a nested structure described at https://github.com/nyphilarchive/PerformanceHistory. The outermost key is `'programs'`.

In [2]:
url = 'https://raw.githubusercontent.com/nyphilarchive/PerformanceHistory/master/Programs/json/complete.json'

r = requests.get(url)
programs = r.json()['programs']

Each program consists of a list of works that is performed at multiple concerts. Using `pd.json_normalize`, we can load the details for each work and concert into separate data frames. Additionally, each work has soloists listed, which can be separated into a third dataframe.

In [3]:
meta_cols = ['id', 'programID', 'orchestra', 'season']
df_concerts = pd.json_normalize(data=programs, record_path='concerts',
    meta=meta_cols)
df_works = pd.json_normalize(data=programs, record_path='works',
    meta=meta_cols)
df_soloists = pd.json_normalize(data=programs, record_path=['works', 'soloists'],
    meta=meta_cols)

In [4]:
df_works.head()

,ID,composerName,workTitle,conductorName,soloists,movement,interval,movement._,movement.em,workTitle._,workTitle.em,id,programID,orchestra,season
0,52446*,"Beethoven, Ludwig van","SYMPHONY NO. 5 IN C MINOR, OP.67","Hill, Ureli Corelli",[],NaN,NaN,NaN,NaN,NaN,NaN,00646b9f-fec7-4ffb-9fb1-faae410bd9dc-0.1,3853,New York Philharmonic,1842-43
1,8834*4,"Weber, Carl Maria Von",OBERON,"Timm, Henry C.","[{'soloistName': 'Otto, Antoinette', 'soloistI...","""Ozean, du Ungeheuer"" (Ocean, thou mighty mons...",NaN,NaN,NaN,NaN,NaN,00646b9f-fec7-4ffb-9fb1-faae410bd9dc-0.1,3853,New York Philharmonic,1842-43
2,3642*,"Hummel, Johann","QUINTET, PIANO, D MINOR, OP. 74",NaN,"[{'soloistName': 'Scharfenberg, William', 'sol...",NaN,NaN,NaN,NaN,NaN,NaN,00646b9f-fec7-4ffb-9fb1-faae410bd9dc-0.1,3853,New York Philharmonic,1842-43
3,0*,NaN,NaN,NaN,[],NaN,Intermission,NaN,NaN,NaN,NaN,00646b9f-fec7-4ffb-9fb1-faae410bd9dc-0.1,3853,New York Philharmonic,1842-43
4,8834*3,"Weber, Carl Maria Von",OBERON,"Etienne, Denis G.",[],Overture,NaN,NaN,NaN,NaN,NaN,00646b9f-fec7-4ffb-9fb1-faae410bd9dc-0.1,3853,New York Philharmonic,1842-43


In [5]:
df_soloists.head()

,soloistName,soloistInstrument,soloistRoles,id,programID,orchestra,season
0,"Otto, Antoinette",Soprano,S,00646b9f-fec7-4ffb-9fb1-faae410bd9dc-0.1,3853,New York Philharmonic,1842-43
1,"Scharfenberg, William",Piano,A,00646b9f-fec7-4ffb-9fb1-faae410bd9dc-0.1,3853,New York Philharmonic,1842-43
2,"Hill, Ureli Corelli",Violin,A,00646b9f-fec7-4ffb-9fb1-faae410bd9dc-0.1,3853,New York Philharmonic,1842-43
3,"Derwort, G. H.",Viola,A,00646b9f-fec7-4ffb-9fb1-faae410bd9dc-0.1,3853,New York Philharmonic,1842-43
4,"Boucher, Alfred",Cello,A,00646b9f-fec7-4ffb-9fb1-faae410bd9dc-0.1,3853,New York Philharmonic,1842-43


We first outer merge the concert and works datasets to construct a single `DataFrame`.

In [6]:
df = pd.merge(df_works, df_concerts, 'outer', on=meta_cols)

In [7]:
df.head()

,ID,composerName,workTitle,conductorName,soloists,movement,interval,movement._,movement.em,workTitle._,workTitle.em,id,programID,orchestra,season,eventType,Location,Venue,Date,Time
0,52446*,"Beethoven, Ludwig van","SYMPHONY NO. 5 IN C MINOR, OP.67","Hill, Ureli Corelli",[],NaN,NaN,NaN,NaN,NaN,NaN,00646b9f-fec7-4ffb-9fb1-faae410bd9dc-0.1,3853,New York Philharmonic,1842-43,Subscription Season,"Manhattan, NY",Apollo Rooms,1842-12-07T05:00:00Z,8:00PM
1,8834*4,"Weber, Carl Maria Von",OBERON,"Timm, Henry C.","[{'soloistName': 'Otto, Antoinette', 'soloistI...","""Ozean, du Ungeheuer"" (Ocean, thou mighty mons...",NaN,NaN,NaN,NaN,NaN,00646b9f-fec7-4ffb-9fb1-faae410bd9dc-0.1,3853,New York Philharmonic,1842-43,Subscription Season,"Manhattan, NY",Apollo Rooms,1842-12-07T05:00:00Z,8:00PM
2,3642*,"Hummel, Johann","QUINTET, PIANO, D MINOR, OP. 74",NaN,"[{'soloistName': 'Scharfenberg, William', 'sol...",NaN,NaN,NaN,NaN,NaN,NaN,00646b9f-fec7-4ffb-9fb1-faae410bd9dc-0.1,3853,New York Philharmonic,1842-43,Subscription Season,"Manhattan, NY",Apollo Rooms,1842-12-07T05:00:00Z,8:00PM
3,0*,NaN,NaN,NaN,[],NaN,Intermission,NaN,NaN,NaN,NaN,00646b9f-fec7-4ffb-9fb1-faae410bd9dc-0.1,3853,New York Philharmonic,1842-43,Subscription Season,"Manhattan, NY",Apollo Rooms,1842-12-07T05:00:00Z,8:00PM
4,8834*3,"Weber, Carl Maria Von",OBERON,"Etienne, Denis G.",[],Overture,NaN,NaN,NaN,NaN,NaN,00646b9f-fec7-4ffb-9fb1-faae410bd9dc-0.1,3853,New York Philharmonic,1842-43,Subscription Season,"Manhattan, NY",Apollo Rooms,1842-12-07T05:00:00Z,8:00PM


## 2.2 - Soloists (ignore for now)

At some point it might be useful to replace the `soloists` column of the merged dataframe with meaningful information from `df_soloists`. However, for now we will just drop the column.

In [8]:
df = df.drop(columns='soloists')

# 3 - Selecting a relevant subset of the data
The dataset contains information about all sorts of New York Philharmonic-affiliated events, including lectures and chamber music performances. We will limit our focus to large orchestra works and performances.

## 3.1 - Orchestra and event type

Let's examine the possible values for the `orchestra` and `eventType` columns:

In [9]:
df['orchestra'].value_counts()

New York Philharmonic                       92815
Stadium-NY Philharmonic                     13430
New York Symphony                            9411
Musicians from the New York Philharmonic     3223
Members of NY Philharmonic                   1694
NYP Presentation                             1198
NY Philharmonic Ensembles                    1141
New/National Symphony Orchestra               490
Strike Orchestra (Philharmonic)                39
Shanghai Orchestra Academy                     20
Members of NY Symphony                         11
None                                            3
Name: orchestra, dtype: int64

In [10]:
df['eventType'].value_counts().head(10) 
# and there are many more categories - 60 total

Subscription Season       65375
Stadium Concert           13504
Tour                      12877
Young People's Concert     6072
Promenade                  3479
Special                    2718
Parks                      2562
Chamber                    2220
Student Concert            2075
Runout                     1595
Name: eventType, dtype: int64

The definitions for these categories are available at https://archives.nyphil.org/index.php/help-performancehistory. These help us identify which values most likely represent a full orchestra performance. It is clear to keep values like "New York Philharmonic" and "Subscription Season," but others are ambiguous. For example, the description of "Members of NY Philharmonic" explains that these concerts are "offered to" all members of the orchestra, while the concerts from "Musicians of the NY Philharmonic" are more likely to be chamber performances. What kind of concerts correspond to each group?

In [11]:
df['eventType'][df['orchestra'] == 'Members of NY Philharmonic'].value_counts().head()

Carnegie Pop Concert    688
Special                 602
Festival                131
Student Concert          74
Subscription Season      56
Name: eventType, dtype: int64

In [12]:
df['eventType'][df['orchestra'] == 'Musicians from the New York Philharmonic'].value_counts().head()

Chamber                        657
Holiday Brass                  485
Very Young People's Concert    458
Chamber Concert (Private)      424
Tour - Chamber                 246
Name: eventType, dtype: int64

It is clear that we should keep "Members of NY Philharmonic", but not "Musicians from the New York Philharmonic". We arrive at the final listing of orchestra types to keep:

In [13]:
orch_types = ['New York Philharmonic', 'New York Symphony', 
            'Stadium-NY Philharmonic', 'Members of NY Philharmonic',
            'New/National Symphony Orchestra', 'Strike Orchestra (Philharmonic)']
df = df[df['orchestra'].isin(orch_types)]

Likewise, we start from the full list of event types and discard small events based on the descriptions available from the NY Philharmonic.

In [14]:
event_types = df['eventType'].unique()
event_types = [event for event in event_types if 'Chamber' not in event]
other_small_events = ['Contact!', 'Holiday Brass', 'Insight Series', 
                      'Leinsdorf Lecture', 'Nightcap', 'Off the Grid',
                      'Pre-Concert Recital', 'Sound ON', 
                      'Tour - Very Young People\'s Concert',
                      'Very Young People\'s Concert']
event_types = [event for event in event_types if event not in other_small_events]
df = df[df['eventType'].isin(event_types)]

# 4 - Clean columns

## 4.1 - Check missing values

Check the proportion of missing values in each column:

In [15]:
df.isna().sum() / len(df)

ID               0.000025
composerName     0.152055
workTitle        0.152080
conductorName    0.170726
movement         0.713601
interval         0.847971
movement._       0.998810
movement.em      0.997272
workTitle._      0.999975
workTitle.em     0.999975
id               0.000000
programID        0.000000
orchestra        0.000000
season           0.000000
eventType        0.000000
Location         0.000000
Venue            0.000000
Date             0.000000
Time             0.000000
dtype: float64

There are a significant number of columns with missing values. Not every work is split into movements, so having missing values in the movement column is expected. The interval column, as we see below, appears to only indicate that a program has an intermission, and is not of much use. We can drop the column to greatly reduce the number of missing values in the `composerName`, `workTitle`, and `conductorName` columns.

In [16]:
df.interval.value_counts(normalize=True)

Intermission           0.994857
Intermission-Short     0.003019
Intermission-Second    0.002068
Intermission-Third     0.000056
Name: interval, dtype: float64

In [17]:
df = df[df.interval.isna()].drop(columns='interval')
df.isna().sum() / len(df)

ID               0.000030
composerName     0.000030
workTitle        0.000060
conductorName    0.022049
movement         0.662254
movement._       0.998597
movement.em      0.996783
workTitle._      0.999970
workTitle.em     0.999970
id               0.000000
programID        0.000000
orchestra        0.000000
season           0.000000
eventType        0.000000
Location         0.000000
Venue            0.000000
Date             0.000000
Time             0.000000
dtype: float64

## 4.2 - `'.em'` and `'._`' columns

We notice a few strangely named columns with many missing values (e.g. `'movement.em'`). Can we move this information into the `'movement'` and `'workTitle'` columns? Upon examining some representative rows with information in these columns, I figured out that the `'.em'` column contains italicized text in the title of the work (see for example ID 8897*, where [Carmen is italicized in the program](https://archives.nyphil.org/index.php/artifact/7fa203d8-1167-4ec9-b2b0-11a45b02a4a7-0.1) (click "Show all")). This probably came from an `<em>` HTML tag.

In [18]:
cols = ['movement._', 'movement.em', 'workTitle._', 'workTitle.em']
df[df[cols].notna().any(axis=1)].sample(5, random_state=3)

,ID,composerName,workTitle,conductorName,movement,movement._,movement.em,workTitle._,workTitle.em,id,programID,orchestra,season,eventType,Location,Venue,Date,Time
15531,8867*2,"Sibelius, Jean","LEMMINKAINEN SUITE (LEGENDS), OP. 22","Stransky, Josef",NaN,NaN,The Swan of Tuonela,NaN,NaN,934e8e20-953f-4a30-aa1e-c65c24f80b61-0.1,7013,New York Philharmonic,1918-19,Runout,"Wilkes-Barre, PA",Irem Temple,1919-02-03T05:00:00Z,8:30PM
82859,8867*4,"Sibelius, Jean","LEMMINKAINEN SUITE (LEGENDS), OP. 22","Jarvi (Järvi), Neeme",NaN,NaN,Lemminkäinen's Return,NaN,NaN,14e4f1ac-ddfb-451f-b74e-456002ce84a5-0.1,5777,New York Philharmonic,1979-80,Subscription Season,"Manhattan, NY",Avery Fisher Hall,1980-02-21T05:00:00Z,8:00PM
37861,8867*4,"Sibelius, Jean","LEMMINKAINEN SUITE (LEGENDS), OP. 22","Barbirolli , John",NaN,NaN,Lemminkäinen's Return,NaN,NaN,d78eb0cf-f5d4-4ef4-aa39-ee67360f1f7c-0.1,4936,New York Philharmonic,1938-39,Runout,"Princeton, NJ",McCarter Theatre,1938-10-29T05:00:00Z,8:30PM
108570,5696*1,"Williams, John",HARRY POTTER: SUITE,"Williams, John",NaN,Hedwig's Theme (from ),Harry Potter and the Sorcerer's Stone,NaN,NaN,e938cda9-a161-4d18-9034-c2a13eb64cde-0.1,9746,New York Philharmonic,2007-08,Subscription Season,"Manhattan, NY",Avery Fisher Hall,2007-09-14T04:00:00Z,8:00PM
47208,8867*2,"Sibelius, Jean","LEMMINKAINEN SUITE (LEGENDS), OP. 22","Stokowski, Leopold",NaN,NaN,The Swan of Tuonela,NaN,NaN,73c4d118-6bd8-4771-b97d-a36e9956b740-0.1,6408,New York Philharmonic,1946-47,Subscription Season,"Manhattan, NY",Carnegie Hall,1946-12-27T05:00:00Z,2:30PM


It won't be feasible to reconstruct the exact work title, but let's concatenate the strings in the two columns and impute it into the non-suffixed columns.
 
Is the `'movement'` column always empty when the other two have values present (and likewise for '`workTitle'`)? If so, the following code should print four `0`s.

In [19]:
for col in ['movement', 'workTitle']:
    for suffix in ['.em', '._']:
        print(df[df[col+suffix].notna()][col].notna().sum())

0
0
0
0


It is safe to consolidate these columns. Let's do that and drop the `.em` and `._` suffixed columns.

In [20]:
for col in ['movement', 'workTitle']:
    rows = df[col].isna()
    df[col][rows] = df[col+'._'][rows] + ' ' +  df[col+'.em'][rows]
    df.drop(columns=[col+'._', col+'.em'], inplace=True)

In [21]:
df.isna().sum()

ID                   3
composerName         3
workTitle            3
conductorName     2200
movement         65939
id                   0
programID            0
orchestra            0
season               0
eventType            0
Location             0
Venue                0
Date                 0
Time                 0
dtype: int64

## 4.3 - Remaining missing values
Many works are not comprised of multiple movements, so we can keep the missing values in the movement column.

There are many instances where a conductor is not listed. Since most large-ensemble works are performed with a conductor, let's check the rows associated with these missing values.

In [22]:
df[df['conductorName'].isna()].sample(5, random_state=0)

,ID,composerName,workTitle,conductorName,movement,id,programID,orchestra,season,eventType,Location,Venue,Date,Time
97957,8761*3,"Bach, Johann Sebastian","PARTITA NO. 2, D MINOR, BWV 1004",NaN,Sarabande,ea5bfd58-8319-4563-9b07-2fc5388d7062-0.1,8015,New York Philharmonic,1996-97,Subscription Season,"Manhattan, NY",Avery Fisher Hall,1997-01-15T05:00:00Z,8:00PM
100240,8113*3,"Bach, Johann Sebastian","SONATA, VIOLIN, UNACCOMPANIED, NO. 2, A MINOR,...",NaN,Andante,b9a40b38-97a7-4aa3-8fb1-ed62ff9bb1ea-0.1,7996,New York Philharmonic,1998-99,Subscription Season,"Manhattan, NY",Avery Fisher Hall,1999-03-20T05:00:00Z,8:00PM
96695,3152*17,"Gabrieli, Giovanni","SACRAE SYMPHONIAE, BOOK 1",NaN,"Sonata pian' e forte a 8, C. 175 (arr. Fracken...",56290366-ef46-4af4-a24c-f04f9251b5d2-0.1,7808,New York Philharmonic,1995-96,Subscription Season,"Manhattan, NY",Avery Fisher Hall,1995-11-09T05:00:00Z,8:00PM
6416,10359*,"Tchaikovsky, Pyotr Ilyich","HUMORESQUE (SOLO PIANO), OP. 10, NO. 2",NaN,NaN,8a73c52f-95a8-411c-b6e8-3d7bc4d7b90a-0.1,10525,New York Symphony,1904-05,Special,"Manhattan, NY",Carnegie Hall,1905-01-18T05:00:00Z,8:15PM
32935,11706*,"Traditional,",DANCE OF THE COCONUT GROVE (ARR. Hairston),NaN,NaN,963ccf25-b762-4265-b276-26f8616ff660-0.1,13124,Stadium-NY Philharmonic,1932-33,Stadium Concert,"Manhattan, NY",Lewisohn Stadium,1933-08-18T04:00:00Z,8:30PM


It looks like these rows represent smaller ensemble works performed without a conductor. Although possible that some rows will represent orchestral works, with the conductor information missing, we will not lose much by dropping these rows.

In [23]:
df = df[df['conductorName'].notna()]

After this operation, we are left with only missing movement information (as expected).

In [24]:
df.isna().sum()

ID                   0
composerName         0
workTitle            0
conductorName        0
movement         64576
id                   0
programID            0
orchestra            0
season               0
eventType            0
Location             0
Venue                0
Date                 0
Time                 0
dtype: int64

## 4.4 - Date and Time
It may be useful to know the date and time of a performance, but the time zone doesn't really matter. Let's create a single DateTime column with `datetime` objects, throwing away the time zone and placing performances with no indicated time at midnight. We then drop the original date and time columns.

In [25]:
df['Date'].sample()

121507    2018-03-17T04:00:00Z
Name: Date, dtype: object

In [26]:
# check that all dates split into two parts on the 'T'
(df['Date'].str.split('T').str.len() == 2).all()

True

In [27]:
df['DateTime'] = pd.to_datetime(df['Date'].str.split('T').str[0] \
                                 + ' ' + df['Time'].str.replace('None', ''))
df.drop(columns=['Date', 'Time'], inplace=True)
df['DateTime'].sample(5)

37517    1938-07-07 20:30:00
112081   2010-03-15 19:30:00
92846    1991-01-09 20:00:00
5346     1901-07-06 15:00:00
8214     1910-03-18 20:15:00
Name: DateTime, dtype: datetime64[ns]

## 4.5 - Movements
Some works are separated into multiple rows by movement, but not all multi-movement works are separated in this way. For example, Tchaikovsky's _Symphony No. 4_ (relevant row shown below) is a four-movement work, but is reported as a single row.

In [28]:
df[df['workTitle'].str.lower().str.contains('symphony')].sample(random_state=0)

,ID,composerName,workTitle,conductorName,movement,id,programID,orchestra,season,eventType,Location,Venue,DateTime
12598,50064*,"Tchaikovsky, Pyotr Ilyich","SYMPHONY NO. 4, F MINOR, OP. 36","Stransky, Josef",NaN,e4cc1030-9497-4694-83bd-8322dd465305-0.1,2113,New York Philharmonic,1915-16,Tour,"Ithaca, NY",Bailey Hall,1916-02-19 20:15:00


It is unclear which works will have movements reported, so let's represent each work as only a single row.

The column `ID` is in the format (work ID)*(movement ID), so we can use this to group by work. First let's separate this into two columns:

In [29]:
df['work_id'] = df['ID'].str.split('*').str[0]
df['mvt_id'] = df['ID'].str.split('*').str[1]

Now group by `DateTime` and `work_id` (to get a unique performance of the work) and count the number of unique movement IDs.

In [30]:
grouped = df.groupby(['DateTime', 'work_id'])['mvt_id'].nunique()
grouped

DateTime             work_id
1842-12-07 20:00:00  52446      1
                     5543       1
                     8336       1
                     8834       2
                     8835       1
                               ..
2020-03-07 20:00:00  53102      1
2020-03-10 19:30:00  1490       1
                     52638      1
                     52644      1
                     53102      1
Name: mvt_id, Length: 85433, dtype: int64

Let's check if this worked:

In [31]:
grouped[grouped>1].sample(random_state=0)

DateTime             work_id
1950-03-16 20:45:00  9006       3
Name: mvt_id, dtype: int64

In [32]:
df[df['DateTime']==pd.to_datetime('1950-03-16 20:45:00')][df['work_id']=='9006']

C:\Users\Brian\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,ID,composerName,workTitle,conductorName,movement,id,programID,orchestra,season,eventType,Location,Venue,DateTime,work_id,mvt_id
50601,9006*17,"Wagner, Richard",GOTTERDAMMERUNG [GÖTTERDÄMMERUNG],"De Sabata, Victor",PROLOGUE: Siegfried's Rhine Journey,830a1bc8-16ed-434c-a282-43d851cfc5c4-0.1,3801,New York Philharmonic,1949-50,Subscription Season,"Manhattan, NY",Carnegie Hall,1950-03-16 20:45:00,9006,17
50603,9006*21,"Wagner, Richard",GOTTERDAMMERUNG [GÖTTERDÄMMERUNG],"De Sabata, Victor",Siegfried's Death (orchestra without singer - ...,830a1bc8-16ed-434c-a282-43d851cfc5c4-0.1,3801,New York Philharmonic,1949-50,Subscription Season,"Manhattan, NY",Carnegie Hall,1950-03-16 20:45:00,9006,21
50605,9006*10,"Wagner, Richard",GOTTERDAMMERUNG [GÖTTERDÄMMERUNG],"De Sabata, Victor","Siegfried's Funeral Music, ACT III, scene ii",830a1bc8-16ed-434c-a282-43d851cfc5c4-0.1,3801,New York Philharmonic,1949-50,Subscription Season,"Manhattan, NY",Carnegie Hall,1950-03-16 20:45:00,9006,10


_Götterdämmerung_ by Wagner is a [multi-movement opera](https://en.wikipedia.org/wiki/G%C3%B6tterd%C3%A4mmerung), but only 3 movements were performed at the [concert](https://archives.nyphil.org/index.php/artifact/830a1bc8-16ed-434c-a282-43d851cfc5c4-0.1).

Now, we update the dataframe. We drop the `movement` column and drop rows that have the same `DateTime` and `work_id`. The resulting dataframe should be the same length as the `grouped` Series.

In [33]:
df = df.drop(columns=['movement', 'mvt_id'])\
        .drop_duplicates(['DateTime', 'work_id'])

len(df), len(grouped)

(85433, 85433)

We can finally add the grouped series as a new column with the number of movements.

In [34]:
df = df.join(grouped, on=['DateTime', 'work_id'])

## 4.6 - Reorder and rename columns for readability

In [35]:
df = df.rename(columns={'id': 'program_guid', 'programID': 'program_id',
                       'mvt_id': 'num_movements'})

cols = ['program_guid', 'program_id', 'work_id', 'composerName', 'workTitle', 
       'num_movements', 'orchestra', 'conductorName', 'season', 'eventType', 'Location', 
        'Venue', 'DateTime']

df = df[cols]

In [36]:
df.head()

,program_guid,program_id,work_id,composerName,workTitle,num_movements,orchestra,conductorName,season,eventType,Location,Venue,DateTime
0,00646b9f-fec7-4ffb-9fb1-faae410bd9dc-0.1,3853,52446,"Beethoven, Ludwig van","SYMPHONY NO. 5 IN C MINOR, OP.67",1,New York Philharmonic,"Hill, Ureli Corelli",1842-43,Subscription Season,"Manhattan, NY",Apollo Rooms,1842-12-07 20:00:00
1,00646b9f-fec7-4ffb-9fb1-faae410bd9dc-0.1,3853,8834,"Weber, Carl Maria Von",OBERON,2,New York Philharmonic,"Timm, Henry C.",1842-43,Subscription Season,"Manhattan, NY",Apollo Rooms,1842-12-07 20:00:00
5,00646b9f-fec7-4ffb-9fb1-faae410bd9dc-0.1,3853,8835,"Rossini, Gioachino",ARMIDA,1,New York Philharmonic,"Timm, Henry C.",1842-43,Subscription Season,"Manhattan, NY",Apollo Rooms,1842-12-07 20:00:00
6,00646b9f-fec7-4ffb-9fb1-faae410bd9dc-0.1,3853,8837,"Beethoven, Ludwig van","FIDELIO, OP. 72",1,New York Philharmonic,"Timm, Henry C.",1842-43,Subscription Season,"Manhattan, NY",Apollo Rooms,1842-12-07 20:00:00
7,00646b9f-fec7-4ffb-9fb1-faae410bd9dc-0.1,3853,8336,"Mozart, Wolfgang Amadeus","ABDUCTION FROM THE SERAGLIO,THE, K.384",1,New York Philharmonic,"Timm, Henry C.",1842-43,Subscription Season,"Manhattan, NY",Apollo Rooms,1842-12-07 20:00:00


# 5 - Analysis

In [37]:
df.isna().sum()

program_guid     0
program_id       0
work_id          0
composerName     0
workTitle        0
num_movements    0
orchestra        0
conductorName    0
season           0
eventType        0
Location         0
Venue            0
DateTime         0
dtype: int64

In [38]:
df['orchestra'].value_counts()

New York Philharmonic              67044
Stadium-NY Philharmonic             9358
New York Symphony                   7326
Members of NY Philharmonic          1288
New/National Symphony Orchestra      387
Strike Orchestra (Philharmonic)       30
Name: orchestra, dtype: int64

In [39]:
df['composerName'].value_counts()[:10]

Beethoven,  Ludwig  van        6050
Wagner,  Richard               4927
Tchaikovsky,  Pyotr  Ilyich    4093
Mozart,  Wolfgang  Amadeus     3948
Brahms,  Johannes              3334
Strauss,  Richard              2321
Bach,  Johann  Sebastian       1728
Mendelssohn,  Felix            1645
Ravel,  Maurice                1644
Dvorak,  Antonín               1596
Name: composerName, dtype: int64

In [40]:
df['composerName'].value_counts().hist()

# Ideas:
- Are works of some composers more likely to be programmed together?
- Are works from lesser-known composers programmed alongside those of better-known composers?
- Are on-tour concerts more or less adventurous than NYC concerts?
- Bring in subscribers dataset to get concert attendance by subscribers (but not full attendance?) https://archives.nyphil.org/index.php/open-data